In [1]:
import numpy as np
import pandas as pd
import os
import xlrd
import re

In [2]:
file = 'RDO No. 1 - Laoag City, Ilocos Norte.xls'
test = pd.ExcelFile(f'data/{file}')

In [3]:
sheet_names = sorted([name for name in test.sheet_names if name.strip().lower().startswith('sheet')], key=lambda name: int(re.search(r'\d+', name).group()))
last_sheet_name = sheet_names[-1] if sheet_names else None

In [4]:
last_sheet_name

'Sheet 9 (DO 047-2023)'

In [5]:
df = pd.read_excel("data/"+file, sheet_name=last_sheet_name, header=None, skiprows=range(40))

In [6]:
df

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,(original signed),NaN,NaN
3,NaN,BENJAMIN E. DIOKNO,NaN,NaN
4,NaN,Secretary of Finance,NaN,NaN
...,...,...,...,...
25301,TAXES SHALL BE (1) THE FAIR MARKET VALUE ...,NaN,NaN,NaN
25302,(i.e. ZONAL VALUES) OR (2.) THE FAIR MAR...,NaN,NaN,NaN
25303,"PROVINCIAL/CITY/MUNICIPAL ASSESSOR, WHICH...",NaN,NaN,NaN
25304,NaN,NaN,NaN,NaN


In [60]:
# FINAL RN
def test_function():
    print("All conditions met! Function called.\n")

for index, row in df.iterrows():
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    
    # Search for "Province :" pattern
    if re.search(r"province\s*:", combined_row, re.IGNORECASE):
        province_match_flag = True
        print(f"Province match found in row {index}: {combined_row}")
    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        if re.search(r"\b(City|Municipality)\b", next_row, re.IGNORECASE):
            city_match_flag = True
            print(f"City / Municipality match found in row {index + 1}: {next_row}")
    
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        if re.search(r"\b(Barangay|Zone)\b", following_row, re.IGNORECASE):
            barangay_match_flag = True
            print(f"Barangay / Zone match found in row {index + 2}: {following_row}")
    
    # Call the function only if all conditions are true
    if province_match_flag and city_match_flag and barangay_match_flag:
        test_function()
        #then skip 2 rows to make more efficient somehow


City / Municipality match found in row 78: Revenue District Office No. 1 - LAOAG CITY, ILOCOS NORTE
Province match found in row 80: Province :ILOCOS NORTE
City / Municipality match found in row 81: Municipality :LAOAG CITY
Barangay / Zone match found in row 82: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
All conditions met! Function called.

City / Municipality match found in row 120: Revenue District Office No. 1 - LAOAG CITY, ILOCOS NORTE
Province match found in row 122: Province :ILOCOS NORTE
City / Municipality match found in row 123: City / Municipality :LAOAG CITY
Barangay / Zone match found in row 124: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION) -- continuedDO No. 047 - 2023
All conditions met! Function called.

Province match found in row 147: Province :ILOCOS NORTE
City / Municipality match found in row 148: City / Municipality :LAOAG CITY
Barangay / Zone match found in row 149: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION) -- continued
All condit

In [10]:
# at the start try to identify how the sheet is formatted
# 

def test_function():
    print("All conditions met! Function called.\n")

count = 0
for index, row in df.iterrows():
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    additional_info_flag = False
    
    # Search for "Province :" pattern
    if re.search(r"province\s*", combined_row, re.IGNORECASE):
        province_match_flag = True
        print(f"Province match found in row {index}: {combined_row}")
    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        if re.search(r"\b(City|Municipality)\b", next_row, re.IGNORECASE):
            city_match_flag = True
            print(f"City / Municipality match found in row {index + 1}: {next_row}")
    
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        if re.search(r"\b(Barangay|Zone)\b", following_row, re.IGNORECASE):
            barangay_match_flag = True
            print(f"Barangay / Zone match found in row {index + 2}: {following_row}")
                
    # Check specific columns for additional information in the next 3 rows
    for offset in range(3, 6):
        if index + offset < len(df):
            subsequent_row = df.iloc[index + offset]
            
            # Check each column independently
            first_column_match = re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", str(subsequent_row.iloc[0]), re.IGNORECASE)
            second_column_match = re.search(r"V\s*I\s*C\s*I\s*N\s*I\s*T\s*Y|VICINITY", str(subsequent_row.iloc[1]), re.IGNORECASE)
            third_column_match = re.search(r"C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", str(subsequent_row.iloc[2]), re.IGNORECASE | re.DOTALL)
            fourth_column_match = re.search(r"REV\s*ZV\s*/\s*SQM", str(subsequent_row.iloc[3]), re.IGNORECASE)

            # Set the flag if all columns match the criteria
            if first_column_match and second_column_match and third_column_match and fourth_column_match:
                count += 1
                additional_info_flag = True
                print(f"Additional information match found in row {index + offset}")
                break  # No need to check further if a match is found

    # Call the function only if all conditions are true
    if province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag:
        test_function()

print(f'count: {count}')

City / Municipality match found in row 78: Revenue District Office No. 1 - LAOAG CITY, ILOCOS NORTE
Additional information match found in row 84
Province match found in row 80: Province :ILOCOS NORTE
City / Municipality match found in row 81: Municipality :LAOAG CITY
Barangay / Zone match found in row 82: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
Additional information match found in row 84
All conditions met! Function called.

Additional information match found in row 84
City / Municipality match found in row 120: Revenue District Office No. 1 - LAOAG CITY, ILOCOS NORTE
Additional information match found in row 126
Province match found in row 122: Province :ILOCOS NORTE
City / Municipality match found in row 123: City / Municipality :LAOAG CITY
Barangay / Zone match found in row 124: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION) -- continuedDO No. 047 - 2023
Additional information match found in row 126
All conditions met! Function called.

Additional inform

In [7]:
def test_function():
    print("All conditions met! Function called.\n")

index = 0
count = 0
while index < len(df):
    row = df.iloc[index]
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    additional_info_flag = False
    
    # Search for "Province :" pattern
    if re.search(r"province\s*:", combined_row, re.IGNORECASE):
        province_match_flag = True
        print(f"Province match found in row {index}: {combined_row}")
    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        if re.search(r"\b(City|Municipality)\b", next_row, re.IGNORECASE):
            city_match_flag = True
            print(f"City / Municipality match found in row {index + 1}: {next_row}")
    
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        if re.search(r"\b(Barangay|Zone)\b", following_row, re.IGNORECASE):
            barangay_match_flag = True
            print(f"Barangay / Zone match found in row {index + 2}: {following_row}")
            
    if province_match_flag and city_match_flag and barangay_match_flag:
        # Check specific columns for additional information in the next 3 rows
        for offset in range(3, 6):
            if index + offset < len(df):
                subsequent_row = df.iloc[index + offset]
                
                # Check each column independently
                first_column_match = re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", str(subsequent_row.iloc[0]), re.IGNORECASE)
                second_column_match = re.search(r"V\s*I\s*C\s*I\s*N\s*I\s*T\s*Y|VICINITY", str(subsequent_row.iloc[1]), re.IGNORECASE)
                third_column_match = re.search(r"C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", str(subsequent_row.iloc[2]), re.IGNORECASE | re.DOTALL)
                fourth_column_match = re.search(r"REV\s*ZV\s*/\s*SQM", str(subsequent_row.iloc[3]), re.IGNORECASE)
    
                # Set the flag if all columns match the criteria
                if first_column_match and second_column_match and third_column_match and fourth_column_match:
                    additional_info_flag = True
                    print(f"Additional information match found in row {index + offset}")
                    # Skip to the row after the current match
                    index += offset
                    break  # No need to check further if a match is found

    # Call the function only if all conditions are true
    # if province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag:
    #     test_function()
    
    if (province_match_flag or city_match_flag or barangay_match_flag or additional_info_flag) and not (province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag):
        print(f"Condition check in row {index}:")
        print(f"  Province match: {'True' if province_match_flag else 'False'}")
        print(f"  City/Municipality match: {'True' if city_match_flag else 'False'}")
        print(f"  Barangay/Zone match: {'True' if barangay_match_flag else 'False'}")
        print(f"  Additional information match: {'True' if additional_info_flag else 'False'}")
        count += 1
        print(count)

    index += 1

City / Municipality match found in row 78: Revenue District Office No. 1 - LAOAG CITY, ILOCOS NORTE
Condition check in row 77:
  Province match: False
  City/Municipality match: True
  Barangay/Zone match: False
  Additional information match: False
1
Province match found in row 80: Province :ILOCOS NORTE
City / Municipality match found in row 81: Municipality :LAOAG CITY
Barangay / Zone match found in row 82: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
Additional information match found in row 84
City / Municipality match found in row 120: Revenue District Office No. 1 - LAOAG CITY, ILOCOS NORTE
Condition check in row 119:
  Province match: False
  City/Municipality match: True
  Barangay/Zone match: False
  Additional information match: False
2
Province match found in row 122: Province :ILOCOS NORTE
City / Municipality match found in row 123: City / Municipality :LAOAG CITY
Barangay / Zone match found in row 124: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION) 

In [67]:
df.loc[24857]

0    STREET NAME / SUBDIVISION / CONDOMINIUM
1                                   VICINITY
2                           CLASSIFI\nCATION
3             4th REV\nZV / SQM\n___________
Name: 24857, dtype: object

In [68]:
testtest = df.loc[24857]

In [70]:
testtest.iloc[0]

'STREET NAME / SUBDIVISION / CONDOMINIUM'

In [77]:
re.search(r"C\s*L\s*A\s*S\s*S\s*I\s*F\s*I\s*C\s*A\s*T\s*I\s*O\s*N", str(testtest.iloc[2]), re.IGNORECASE)

<re.Match object; span=(0, 15), match='CLASSIFI\nCATION'>

In [38]:
import re

joined_row = "Country: Philippines, Province : Quezon, City: Quezon City"

pattern = r"province\s*:"

match = re.search(pattern, joined_row, re.IGNORECASE)

if match:
    print("Match found:", match.group())
else:
    print("No match found.")


Match found: Province :
